In [1]:
import pandas as pd
import cooler
import numpy as np
import bioframe
import cooltools.expected
import cooltools.snipping
import cooltools

In [2]:
cooltools.__version__

'0.2.0'

# Load data

## Pileup positions

In [3]:
url = "https://raw.githubusercontent.com/gerlichlab/ngs/master/testFiles/test3_realData.bed"
positionFrame = pd.read_csv(url, sep="\t", index_col=0)

In [4]:
positionFrame

,chrom,pos
15795,chr17,27571011
860,chr1,47995376
38158,chr9,85942215
11284,chr13,60800839
6265,chr10,123495918
...,...,...
8392,chr11,123132315
30535,chr5,162710426
13067,chr15,44422321
23599,chr3,1159864


## Cooler

In [5]:
%%bash
curl https://raw.githubusercontent.com/gerlichlab/ngs/master/testFiles/test3_realdata.mcool > test.mcool

% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.2M  100 40.2M    0     0  25.0M      0  0:00:01  0:00:01 --:--:-- 25.1M


In [6]:
c = cooler.Cooler("test.mcool::/resolutions/50000")


# Prepare pileup ingredients

## Supports

In [7]:
chromsizes = bioframe.fetch_chromsizes("hg19")
# download centromers
cens = bioframe.fetch_centromeres("hg19")
cens.set_index("chrom", inplace=True)
cens = cens.mid
# define chromosomes that are well defined (filter out unassigned contigs)
GOOD_CHROMS = list(chromsizes.index[:23])
# construct arm regions (for each chromosome fro 0-centromere and from centromere to the end)
arms = [
    arm
    for chrom in GOOD_CHROMS
    for arm in (
        (chrom, 0, cens.get(chrom, 0)),
        (chrom, cens.get(chrom, 0), chromsizes.get(chrom, 0)),
    )
]
# construct dataframe out of arms
arms = pd.DataFrame(arms, columns=["chrom", "start", "end"])

In [8]:
arms.head()

,chrom,start,end
0,chr1,0,125200000
1,chr1,125200000,249250621
2,chr2,0,93650000
3,chr2,93650000,243199373
4,chr3,0,90900000


## Assign regions

In [9]:
binsize = 50000
window = 500000
chroms = positionFrame["chrom"]
positions = positionFrame["pos"]

In [10]:
# construct windows from the passed chromosomes and positions
snipping_windows = cooltools.snipping.make_bin_aligned_windows(
        binsize, chroms.values, positions.values, window
)
# assign chromosomal arm to each position
snipping_windows = cooltools.snipping.assign_regions(
        snipping_windows, list(arms.itertuples(index=False, name=None))
)

In [11]:
snipping_windows.head()

,chrom,start,end,lo,hi,region
0,chr17,27050000,28100000,541,562,chr17:24000000-81195210
1,chr1,47450000,48500000,949,970,chr1:0-125200000
2,chr9,85400000,86450000,1708,1729,chr9:49000000-141213431
3,chr13,60300000,61350000,1206,1227,chr13:17900000-115169878
4,chr10,122950000,124000000,2459,2480,chr10:40150000-135534747


## Expected dataframe

In [12]:
expected = cooltools.expected.diagsum(
        c,
        tuple(arms.itertuples(index=False, name=None)),
        transforms={"balanced": lambda p: p["count"] * p["weight1"] * p["weight2"]},
        ignore_diags=0,
    )
# construct a single dataframe for all regions (arms)
expected_df = pd.concat(
    [
        exp.reset_index().assign(chrom=reg[0], start=reg[1], end=reg[2])
        for reg, exp in expected.items()
    ]
)
# aggregate diagonals over the regions specified by chrom, start, end (arms)
expected_df = (
    expected_df.groupby(["chrom", "start", "end", "diag"])
    .aggregate({"n_valid": "sum", "count.sum": "sum", "balanced.sum": "sum"})
    .reset_index()
)
# account for different number of valid bins in diagonals
expected_df["balanced.avg"] = expected_df["balanced.sum"] / expected_df["n_valid"]

In [13]:
expected_df.head()

,chrom,start,end,diag,n_valid,count.sum,balanced.sum,balanced.avg
0,chr1,0,125200000,0,2122,1858.0,18.949417,0.008930
1,chr1,0,125200000,1,2044,2064.0,23.857389,0.011672
2,chr1,0,125200000,2,2020,1757.0,19.482067,0.009645
3,chr1,0,125200000,3,2006,1539.0,17.003948,0.008477
4,chr1,0,125200000,4,1996,1359.0,14.855496,0.007443


# Pileup

In [14]:
oe_snipper = cooltools.snipping.ObsExpSnipper(c, expected_df)

In [15]:
oe_pileN = cooltools.snipping.pileup(
    snipping_windows, oe_snipper.select, oe_snipper.snip
)

In [19]:
oe_pileN.shape

(21, 21, 100)